In [ ]:
import pandas as pd
import numpy as np
import time
import keplergl as kgl
from dksr import extract_sample_network
from geojson import LineString as geoLS
from shapely.geometry import LineString as shapLS
from shapely.ops import unary_union
from shapely import MultiPoint

In [ ]:
#north, east, south, west = 33.798, -84.378, 33.763, -84.422 

In [ ]:
#berlin
north, east, south, west = 52.460298,13.328991,52.515803,13.455849

In [ ]:
df = extract_sample_network(north, east, south, west, 50, seed=42)

In [ ]:
df.head(10)#.style.background_gradient(cmap='Blues', subset='length_km')

In [ ]:
geo_list = pd.DataFrame(np.zeros((len(df)),dtype=object),columns=['geo_json'])
                            
for i in range(0,len(df)):                       
    z_list = [0] * len(df['coordinates'][i])
    list0 = df['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
        
    geo_list.iloc[i] = [geoLS(list1.tolist())]

In [ ]:
geo_list.iloc[0]

In [ ]:
#OSMnx returns values in lat, lng while KeplerGL expetcs them in lng,lat
def switch_lat_lon(coords):
    return [coords[1], coords[0], coords[2]]

In [ ]:
# Apply the function to the 'coordinates' column using a lambda function
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})

In [ ]:
from IPython.display import display
with pd.option_context('display.max_colwidth', None):
    display(geo_list.iloc[1])

In [ ]:
#a first test plot
#map_0 = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
#map_0

In [ ]:
geo_list.iloc[0]

In [ ]:
# further transform routes data
# 1. interpolate the various LineStrings by shapely LineString interoplate function
# 2. generate timestamps for every interpolated point on a LineString

In [ ]:
def trace_transform(df, speed, distance_delta):
    new_df = pd.DataFrame(columns=['coordinates', 'length_km', 'timestamps_list'])
    
    for index, row in df.iterrows():
        coordinates = row['coordinates']
        length_km = row['length_km']

        # create points by given distance_delta
        line = LineString(coordinates)
        distances = np.arange(0, line.length, distance_delta)
        points = MultiPoint([line.interpolate(distance) for distance in distances])
        lat_lon_values = [[p.x, p.y] for p in points.geoms]

        # partition_length in km
        total_points = len(lat_lon_values)
        partition_length = (length_km / total_points)

        # time distance in seconds
        time_delta = (partition_length / speed) * 3600

        row_timestamps = []

        for i in range(total_points):
            if i == 0:
                # set initial timestamp to current time
                row_timestamps.append(time.time())
            else:
                # calculate the time delta from the previous point
                time_delta = (partition_length / speed) * 3600
                # add the time delta to the previous timestamp
                prev_timestamp = row_timestamps[-1]
                next_timestamp = prev_timestamp + time_delta
                # append the new timestamp to the list
                row_timestamps.append(next_timestamp)

        new_df = new_df.append({'coordinates': lat_lon_values, 'length_km': length_km, 'timestamps_list': row_timestamps},
                               ignore_index=True)

    return new_df

In [ ]:
df = trace_transform(df=df, speed=15, distance_delta=0.0001)

In [ ]:
df.head(5)

In [ ]:
geo_list = pd.DataFrame(np.zeros((len(df)),dtype=object),columns=['geo_json'])
                            
for i in range(0,len(df)):                       
    z_list = [0] * len(df['coordinates'][i])
    list0 = df['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
    list2 = np.insert(list1,3,df['timestamps_list'][i],axis=1)
        
    geo_list.iloc[i] = [geoLS(list2.tolist())]

In [ ]:
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})

In [ ]:
# a second plot
map_1 = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
map_1